## Code to Extract Exponential Smoothing Parameters

***

In [1]:
# load modules
import statsmodels.tsa.holtwinters as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

We need to read in each protected dataset, fit the exponential smoothing models, and extract the parameters for each series. Save these as a dataset, and plot in R.

In [2]:
# results file path
results_path = "../../Outputs/Results/"

In [3]:
train_data = pd.read_csv("../../Data/Train/Clean/m3_monthly_micro_h1.csv", header=None, skiprows=1)
# convert to a list of series, and drop missing values
train_data = [x.dropna() for _, x in train_data.iterrows()]

In [4]:
files = os.listdir("../../Data/Train/Clean/")

In [5]:
files = [f for f in files if ("protected_m3_monthly_micro_h1_" in f or "full_m3" in f)]

In [6]:
files

['full_m3_monthly_micro_clean.csv',
 'protected_m3_monthly_micro_h1_AN_0.25.csv',
 'protected_m3_monthly_micro_h1_AN_0.5.csv',
 'protected_m3_monthly_micro_h1_AN_1.5.csv',
 'protected_m3_monthly_micro_h1_AN_1.csv',
 'protected_m3_monthly_micro_h1_AN_2.csv',
 'protected_m3_monthly_micro_h1_Bottom_0.1.csv',
 'protected_m3_monthly_micro_h1_Bottom_0.2.csv',
 'protected_m3_monthly_micro_h1_Bottom_0.4.csv',
 'protected_m3_monthly_micro_h1_DP_0.1.csv',
 'protected_m3_monthly_micro_h1_DP_1.csv',
 'protected_m3_monthly_micro_h1_DP_10.csv',
 'protected_m3_monthly_micro_h1_DP_20.csv',
 'protected_m3_monthly_micro_h1_DP_4.6.csv',
 'protected_m3_monthly_micro_h1_k-nts-plus_10.csv',
 'protected_m3_monthly_micro_h1_k-nts-plus_15.csv',
 'protected_m3_monthly_micro_h1_k-nts-plus_3.csv',
 'protected_m3_monthly_micro_h1_k-nts-plus_5.csv',
 'protected_m3_monthly_micro_h1_k-nts-plus_7.csv',
 'protected_m3_monthly_micro_h1_k-nts_10.csv',
 'protected_m3_monthly_micro_h1_k-nts_15.csv',
 'protected_m3_monthly_

In [7]:
methods = {"full":[], 
           "AN": [0.25, 0.5, 1, 1.5, 2], 
           "DP": [0.1, 1, 4.6, 10, 20],  
           "k-nts": [3, 5, 7, 10, 15],
           "k-nts-plus": [3, 5, 7, 10, 15]}

In [8]:
ses_dict = {}
des_dict = {}
tes_dict = {}

for method in methods.items():
    method_files = [f for f in files if method[0]+"_" in f]
    
    if method[0] == "full":
        param_files = method_files
        [param_files] = param_files
        
        full_data = pd.read_csv("../../Data/Train/Clean/"+param_files, header=None, skiprows=1)
        full_data = [x.dropna() for _, x in full_data.iterrows()]
        full_data = [pd.Series([i if i >= 1 else 1 for i in x]) for x in full_data]
        full_data = [np.log(x) for x in full_data]
        
        ses_params = []
        ses_levels = []
        des_params = []
        des_levels = []
        des_trends = []
        des_keys = ['smoothing_level', 'smoothing_trend']
        tes_params = []
        tes_levels = []
        tes_trends = []
        tes_seasons = []
        tes_keys = ['smoothing_level', 'smoothing_trend', 'smoothing_seasonal']
        
        for x in full_data:
            SES = sm.ExponentialSmoothing(endog=x)
            DES = sm.ExponentialSmoothing(endog=x, trend="additive")
            TES = sm.ExponentialSmoothing(endog=x, trend="additive", seasonal="additive", seasonal_periods=12)
            ses = SES.fit()
            des = DES.fit()
            tes = TES.fit()
            ses_params.append(ses.params['smoothing_level'])
            ses_levels.append(np.mean(ses.level))
            des_params.append({key:value for key,value in des.params.items() if key in des_keys})
            des_levels.append(np.mean(des.level))
            des_trends.append(np.mean(des.trend))
            tes_params.append({key:value for key,value in tes.params.items() if key in tes_keys})
            tes_levels.append(np.mean(tes.level))
            tes_trends.append(np.mean(tes.trend))
            tes_seasons.append(np.mean(tes.season))
            
        ses_dict['original_none_alpha'] = ses_params
        ses_dict['original_none_level'] = ses_levels
        des_dict['original_none_alpha'] = [x['smoothing_level'] for x in des_params]
        des_dict['original_none_beta'] = [x['smoothing_trend'] for x in des_params]
        des_dict['original_none_level'] = des_levels
        des_dict['original_none_trend'] = des_trends
        tes_dict['original_none_alpha'] = [x['smoothing_level'] for x in tes_params]
        tes_dict['original_none_beta'] = [x['smoothing_trend'] for x in tes_params]
        tes_dict['original_none_gamma'] = [x['smoothing_seasonal'] for x in tes_params]
        tes_dict['original_none_level'] = tes_levels
        tes_dict['original_none_trend'] = tes_trends
        tes_dict['original_none_season'] = tes_seasons
    
    else: 
        for param in method[1]:
            param_files = [f for f in method_files if "_"+str(param)+".csv" in f]
            [param_files] = param_files
            
            full_data = pd.read_csv("../../Data/Train/Clean/"+param_files, header=None, skiprows=1)
            full_data = [x.dropna() for _, x in full_data.iterrows()]
            full_data = [pd.Series([i if i >= 1 else 1 for i in x]) for x in full_data]
            full_data = [np.log(x) for x in full_data]
            
            ses_params = []
            ses_levels = []
            des_params = []
            des_levels = []
            des_trends = []
            tes_params = []
            tes_levels = []
            tes_trends = []
            tes_seasons = []
            
            for x in full_data:
                SES = sm.ExponentialSmoothing(endog=x)
                DES = sm.ExponentialSmoothing(endog=x, trend="additive")
                TES = sm.ExponentialSmoothing(endog=x, trend="additive", seasonal="additive", seasonal_periods=12)
                ses = SES.fit()
                des = DES.fit()
                tes = TES.fit()
                ses_params.append(ses.params['smoothing_level'])
                ses_levels.append(np.mean(ses.level))
                des_params.append({key:value for key,value in des.params.items() if key in des_keys})
                des_levels.append(np.mean(des.level))
                des_trends.append(np.mean(des.trend))
                tes_params.append({key:value for key,value in tes.params.items() if key in tes_keys})
                tes_levels.append(np.mean(tes.level))
                tes_trends.append(np.mean(tes.trend))
                tes_seasons.append(np.mean(tes.season))
            
            ses_dict[method[0]+'_'+str(param)+'_alpha'] = np.array(ses_params)
            ses_dict[method[0]+'_'+str(param)+'_level'] = np.array(ses_levels)
            des_dict[method[0]+'_'+str(param)+'_alpha'] = np.array([x['smoothing_level'] for x in des_params])
            des_dict[method[0]+'_'+str(param)+'_beta'] = np.array([x['smoothing_trend'] for x in des_params])
            des_dict[method[0]+'_'+str(param)+'_level'] = des_levels
            des_dict[method[0]+'_'+str(param)+'_trend'] = des_trends
            tes_dict[method[0]+'_'+str(param)+'_alpha'] = np.array([x['smoothing_level'] for x in tes_params])
            tes_dict[method[0]+'_'+str(param)+'_beta'] = np.array([x['smoothing_trend'] for x in tes_params])
            tes_dict[method[0]+'_'+str(param)+'_gamma'] = np.array([x['smoothing_seasonal'] for x in tes_params])
            tes_dict[method[0]+'_'+str(param)+'_level'] = tes_levels
            tes_dict[method[0]+'_'+str(param)+'_trend'] = tes_trends
            tes_dict[method[0]+'_'+str(param)+'_season'] = tes_seasons

Now save parameter dictionaries as dataframes for analysis in R.

In [9]:
pd.DataFrame(ses_dict).to_csv(results_path+"Model Parameters/ses_params.csv", index=False)

In [10]:
pd.DataFrame(des_dict).to_csv(results_path+"Model Parameters/des_params.csv", index=False)

In [11]:
pd.DataFrame(tes_dict).to_csv(results_path+"Model Parameters/tes_params.csv", index=False)